# 可视化

## 生成Cityscapes数据集的GT框

In [115]:
import os
from pycocotools.coco import COCO

In [116]:
from yolox.data import COCOFormatDataset, ValTransform
data_dir = "datasets/Cityscapes"
val_ann = "cocoAnnotations/cityscapes_val_caronly_cocostyle.json"
test_size = (640, 640)
valdataset = COCOFormatDataset(
    data_dir=data_dir,
    name="leftImg8bit_val",
    json_file=val_ann,
    img_dir="leftImg8bit/val",
    img_size=test_size,
    preproc=ValTransform(legacy=False),
)

loading annotations into memory...
Done (t=0.23s)
creating index...
index created!


In [ ]:
valdataset.annotations

In [118]:
def get_info_from_filepath(path):
    for i,anno in enumerate(valdataset.annotations):
        bboxs,img_size,resize_size,file_path = anno
        if file_path == path:
            return anno
    print('anno not found!')
    return None
anno = get_info_from_filepath('munster/munster_000000_000019_leftImg8bit.png')

In [119]:
import cv2
raw_img = cv2.imread(os.path.join(data_dir,'leftImg8bit/val','munster/munster_000000_000019_leftImg8bit.png'))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(raw_img)

In [ ]:
from visualize import vis_GT
from yolox.data.datasets import Cityscapes_CLASSES,KITTI_CLASSES
img = raw_img
img = vis_GT(img, anno[0], max(anno[1])/640, class_names=Cityscapes_CLASSES)
plt.imshow(img)
plt.imsave("save.png",img)

In [122]:
output_path = 'yolox_da_outputs/GT_outputs'
plt.imsave(os.path.join(output_path,"save.png"),img)

### 生成foggy_cityscapesGT 框

In [ ]:
raw_img_path = 'datasets/Cityscapes/leftImg8bit_foggy/val/munster'
output_path = 'yolox_da_outputs/GT_outputs'
for file in os.listdir(raw_img_path):
    foggy_type = file[-6:-4]
    if foggy_type=='02':
        print(file)
        anno = get_info_from_filepath('munster/'+file)
        if anno == None:
            print(file+' has not anno!')
        else:
            resize_ratio = max(anno[1])/640
            raw_img = cv2.imread(os.path.join(data_dir,'leftImg8bit_foggy/val/munster/',file))
            img = vis_GT(raw_img, anno[0], resize_ratio, class_names=Cityscapes_CLASSES)
            # plt.imshow(img)
            cv2.imwrite(os.path.join(output_path,'munster_foggy',file),img)
            # plt.imsave(os.path.join(output_path,'munster',file),img)
            

### 生成cityscapes GT框


In [ ]:
raw_img_path = 'datasets/Cityscapes/leftImg8bit/val/munster'
output_path = 'yolox_da_outputs/GT_outputs'
for file in os.listdir(raw_img_path):
    print(file[0:-4])
    anno = get_info_from_filepath('munster/'+file[0:-4]+".png")
    if anno == None:
        print(file+' has not anno!')
    else:
        resize_ratio = max(anno[1])/640
        raw_img = cv2.imread(os.path.join(data_dir,'leftImg8bit/val/munster/',file))
        img = vis_GT(raw_img, anno[0], resize_ratio, class_names=KITTI_CLASSES)
        # plt.imshow(img)
        cv2.imwrite(os.path.join(output_path,'munster_clear',file),img)
        # plt.imsave(os.path.join(output_path,'munster_clear',file),img) #RGB BGR

In [ ]:
raw_img_path = 'datasets/Cityscapes/leftImg8bit/val/munster'
output_path = 'yolox_da_outputs/GT_outputs'
for file in os.listdir(raw_img_path):
    print(file[0:-4])
    anno = get_info_from_filepath('munster/'+file[0:-4]+"_foggy_beta_0.02.png")
    if anno == None:
        print(file+' has not anno!')
    else:
        resize_ratio = max(anno[1])/640
        raw_img = cv2.imread(os.path.join(data_dir,'leftImg8bit/val/munster/',file))
        img = vis_GT(raw_img, anno[0], resize_ratio, class_names=Cityscapes_CLASSES)
        # plt.imshow(img)
        cv2.imwrite(os.path.join(output_path,'munster_clear',file),img)
        # plt.imsave(os.path.join(output_path,'munster_clear',file),img) #RGB BGR

## 生成Sim10k数据集的GT框

In [4]:
import os
import xml.etree.ElementTree as ET
from collections import Counter

def parse_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []

    for obj in root.findall('object'):
        name = obj.find('name').text
        objects.append(name)

    return objects

def find_objects_in_xml_files(directory):
    # 支持的文件扩展名
    supported_extensions = ('.xml', '.XML')
    all_objects = []

    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(supported_extensions):
                file_path = os.path.join(root, file)
                objects = parse_xml(file_path)
                all_objects.extend(objects)

    return all_objects

def main():
    directory = '/home/sipl/gitspace/yolox_implementation/yolox_damain_adaptation/datasets/Sim10k/Annotations'  # 你的文件夹路径
    all_objects = find_objects_in_xml_files(directory)

    # 计数每个对象出现的次数
    object_counts = Counter(all_objects)

    # 打印每个对象及其计数
    for obj, count in object_counts.items():
        print(f"Object: {obj}, Count: {count}")

if __name__ == '__main__':
    main()


Object: car, Count: 57776
Object: motorbike, Count: 921
Object: person, Count: 4


### voc转coco标注

In [ ]:
import os
import json
import xml.etree.ElementTree as ET

# 定义COCO数据结构
coco_output = {
    "images": [],
    "categories": [],
    "annotations": []
}

# 类别信息，这里只添加了'car'，可以根据实际情况添加更多类别
category_id = 1
coco_output["categories"].append({
    "id": category_id,
    "name": "car"
})

# 遍历Annotations文件夹中的所有XML文件
annotations_dir = 'datasets/Sim10k/Annotations'
for filename in os.listdir(annotations_dir):
    if filename.endswith('.xml'):
        # 解析XML文件
        tree = ET.parse(os.path.join(annotations_dir, filename))
        root = tree.getroot()
        # 获取图片信息
        file_id = int(filename.split('.')[0])
        file_name = filename[:-4] + '.jpg'
        # print(file_name)
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)

        # 添加图片信息到COCO数据结构
        coco_output["images"].append({
            "id": file_id,
            "width": width,
            "height": height,
            "file_name": file_name
        })

        # 遍历所有对象
        for obj in root.iter('object'):
            # 获取对象信息
            class_name = obj.find('name').text
            if class_name not in ['car']:  # 只处理'car'类别
                continue
            truncated = int(obj.find('truncated').text)
            pose = obj.find('pose').text
            difficult = int(obj.find('difficult').text)

            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # 计算面积
            bbox_width = xmax - xmin
            bbox_height = ymax - ymin
            area = bbox_width * bbox_height

            # 添加标注信息到COCO数据结构
            coco_output["annotations"].append({
                "id": len(coco_output["annotations"]) + 1,
                "image_id": file_id,
                "category_id": category_id,
                "segmentation": [],
                "area": area,
                "bbox": [xmin, ymin, bbox_width, bbox_height],
                "iscrowd": 0
            })
        print(filename,"is ok")

# 将COCO数据结构转换为JSON格式并保存到文件
with open('datasets/Sim10k/annotations_coco/sim10k_coco.json', 'w') as output_file:
    json.dump(coco_output, output_file)

print("COCO annotation file has been created.")

In [ ]:
import os
from pycocotools.coco import COCO

from yolox.data import COCOFormatDataset, ValTransform
data_dir = "datasets/Sim10k"
val_ann = "annotations_coco/sim10k_coco.json"
test_size = (640, 640)
valdataset = COCOFormatDataset(
    data_dir=data_dir,
    name="sim10k",
    json_file=val_ann,
    img_dir="JPEGImages",
    img_size=test_size,
    preproc=ValTransform(legacy=False),
)
valdataset.annotations

In [31]:
def get_info_from_filepath(path):
    for i,anno in enumerate(valdataset.annotations):
        bboxs,img_size,resize_size,file_path = anno
        # print(file_path)
        if file_path == path:
            return anno
    print('anno not found!')
    return None
anno = get_info_from_filepath('3399597.jpg')

In [32]:
import cv2
raw_img = cv2.imread(os.path.join(data_dir,'JPEGImages','3399597.jpg'))

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(raw_img)

In [ ]:
from visualize import vis_GT
from yolox.data.datasets import KITTI_CLASSES
img = raw_img
img = vis_GT(img, anno[0], max(anno[1])/640, class_names=KITTI_CLASSES)
plt.imshow(img)
plt.imsave("save.png",img)

### 生成GT图片

In [ ]:
raw_img_path = 'datasets/Sim10k/JPEGImages'
output_path = 'yolox_da_outputs/GT_outputs'
for file in os.listdir(raw_img_path):
        print(file)
        anno = get_info_from_filepath(file)
        if anno == None:
            print(file+' has not anno!')
        else:
            resize_ratio = max(anno[1])/640
            raw_img = cv2.imread(os.path.join(data_dir,'JPEGImages/',file))
            img = vis_GT(raw_img, anno[0], resize_ratio, class_names=KITTI_CLASSES)
            # plt.imshow(img)
            cv2.imwrite(os.path.join(output_path,'sim10k',file),img)
            # plt.imsave(os.path.join(output_path,'munster',file),img)
            

## 生成KITTI数据集的GT框

### voc转coco标注

In [ ]:
import os
import json
import xml.etree.ElementTree as ET

# 定义COCO数据结构
coco_output = {
    "images": [],
    "categories": [],
    "annotations": []
}

# 类别信息，这里只添加了'car'，可以根据实际情况添加更多类别
category_id = 1
coco_output["categories"].append({
    "id": category_id,
    "name": "car"
})

# 遍历Annotations文件夹中的所有XML文件
annotations_dir = 'datasets/KITTI/Annotations'
for filename in os.listdir(annotations_dir):
    if filename.endswith('.xml'):
        # 解析XML文件
        tree = ET.parse(os.path.join(annotations_dir, filename))
        root = tree.getroot()
        # 获取图片信息
        file_id = int(filename.split('.')[0])
        file_name = filename[:-4] + '.png'
        # print(file_name)
        width = int(root.find('size').find('width').text)
        height = int(root.find('size').find('height').text)

        # 添加图片信息到COCO数据结构
        coco_output["images"].append({
            "id": file_id,
            "width": width,
            "height": height,
            "file_name": file_name
        })

        # 遍历所有对象
        for obj in root.iter('object'):
            # 获取对象信息
            class_name = obj.find('name').text
            if class_name not in ['car']:  # 只处理'car'类别
                continue
            truncated = int(obj.find('truncated').text)
            pose = obj.find('pose').text
            difficult = int(obj.find('difficult').text)

            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)

            # 计算面积
            bbox_width = xmax - xmin
            bbox_height = ymax - ymin
            area = bbox_width * bbox_height

            # 添加标注信息到COCO数据结构
            coco_output["annotations"].append({
                "id": len(coco_output["annotations"]) + 1,
                "image_id": file_id,
                "category_id": category_id,
                "segmentation": [],
                "area": area,
                "bbox": [xmin, ymin, bbox_width, bbox_height],
                "iscrowd": 0
            })
        print(filename,"is ok")

# 将COCO数据结构转换为JSON格式并保存到文件
with open('datasets/KITTI/annotations_coco/KITTI_coco.json', 'w') as output_file:
    json.dump(coco_output, output_file)

print("COCO annotation file has been created.")

In [ ]:
import os
from pycocotools.coco import COCO

from yolox.data import COCOFormatDataset, ValTransform
data_dir = "datasets/KITTI"
val_ann = "annotations_coco/KITTI_coco.json"
test_size = (640, 640)
valdataset = COCOFormatDataset(
    data_dir=data_dir,
    name="KITTI",
    json_file=val_ann,
    img_dir="JPEGImages",
    img_size=test_size,
    preproc=ValTransform(legacy=False),
)

valdataset.annotations

In [51]:
def get_info_from_filepath(path):
    for i,anno in enumerate(valdataset.annotations):
        bboxs,img_size,resize_size,file_path = anno
        # print(file_path)
        if file_path == path:
            return anno
    print('anno not found!')
    return None
anno = get_info_from_filepath('000006.png')


In [ ]:
import cv2
raw_img = cv2.imread(os.path.join(data_dir,'JPEGImages','000006.png'))

from matplotlib import pyplot as plt
plt.imshow(raw_img)


In [ ]:
from visualize import vis_GT
from yolox.data.datasets import KITTI_CLASSES
img = raw_img
img = vis_GT(img, anno[0], max(anno[1])/640, class_names=KITTI_CLASSES)
plt.imshow(img)
plt.imsave("save.png",img)

### 生成GT图片

In [ ]:
raw_img_path = 'datasets/KITTI/JPEGImages'
output_path = 'yolox_da_outputs/GT_outputs'
for file in os.listdir(raw_img_path):
        print(file)
        anno = get_info_from_filepath(file)
        if anno == None:
            print(file+' has not anno!')
        else:
            resize_ratio = max(anno[1])/640
            raw_img = cv2.imread(os.path.join(data_dir,'JPEGImages/',file))
            img = vis_GT(raw_img, anno[0], resize_ratio, class_names=KITTI_CLASSES)
            # plt.imshow(img)
            cv2.imwrite(os.path.join(output_path,'KITTI',file),img)
            # plt.imsave(os.path.join(output_path,'munster',file),img)
            